In [13]:
import gmpy2 
import math
from random import randint
from math import gcd
import numpy as np
import libnum
import cv2
from PIL import Image
import timeit
rs = gmpy2.random_state()

In [18]:
class Paillier(object):
    
 
    def __init__(self):
        
        self.publicKey = None
        self.privateKey = None        
        
    def lcm(self, x, y):
        
        return x * y // gcd(x, y) 
    
    def sTn(self,m):
        
        arr = bytes(m, 'utf-8')
        m = int.from_bytes(arr, 'big')
        return m
    
         
    def ifprime(self, num):
        for i in range(2, (num//2 +1)):
            if (num%i) == 0:
                return False
            else:
                return True
     
    def getP(self):
        
        p = gmpy2.mpz_urandomb(rs, 3024)
        
        while not gmpy2.is_prime(p):
            p = p + 1
        return p
    
    def getKeys(self):
        
        p = self.getP()
        q = self.getP() 
        
        n = p * q
        lamda = self.lcm(p-1, q-1)
        
        g = n + 1
        
        mu = (pow(g,lamda,n*n)-1) // n
        mu = libnum.invmod(mu, n)
        
        n = int(n)
        g = int(g)
        lamda = int(lamda)
        mu = int(mu)
        
        self.publicKey = [n, g]
        self.privateKey = [lamda, mu]
        
    
    def encrypt(self, m):
        
        if isinstance(m, str):
            m = self.sTn(m)
        
        #print(m)
        
        n,g = self.publicKey
        r = randint(0,n)
        while gcd(n, r)!= 1:
            r = r + 1
        
        c = (pow(g, m, n*n) * pow(r, n, n*n)) % (n*n)
        
        # print(c)
        return c
    
    def decrypt(self,c):
        
        n, g = self.publicKey
        lamda, mu = self.privateKey
        
        c = (pow(c,lamda, n*n) - 1) // n * mu
        c = c % n
        c = int (c)
        # print(c)

        m = libnum.n2s(c)
        m = m.decode("utf-8") 
        
        return m

        


In [19]:
if __name__ == "__main__":    
    string = 'The public key can be known by everyone, and it is used for encrypting messages. The intention is that messages encrypted with the public key can only be decrypted in a reasonable amount of time by using the private key. The public key is represented by the integers n and e; and, the private key, by the integer d (although n is also used during the decryption process, so it might be considered to be a part of the private key, too). m represents the message (previously prepared with a certain technique explained below).'
    byte = ''
    text = string #byte
    
    start = timeit.default_timer()    
    
    p = Paillier()
    p.getKeys()
    
    
    c = p.encrypt(string) 
    m = p.decrypt(c)
    
    print("Plaintext:", text)
    print("Ciphertext:", c)
    print("Deciphertext: ", m)
    
    stop = timeit.default_timer()

    print('Time for encode and decode: ', stop - start, 's') 
    

Plaintext: The public key can be known by everyone, and it is used for encrypting messages. The intention is that messages encrypted with the public key can only be decrypted in a reasonable amount of time by using the private key. The public key is represented by the integers n and e; and, the private key, by the integer d (although n is also used during the decryption process, so it might be considered to be a part of the private key, too). m represents the message (previously prepared with a certain technique explained below).
Ciphertext: 33571493942968989113141390398766255216227253215780530093547420840389224539351308748641672615489619187848532660078119405911772770962293324353469728439934649219118279573169199569310843520293885038065943702706366974469473447678495494696768254424675977132709732477197072546592518278247505840065878913499618386336173135507287783163855414362493831791734333192671971036907762564987107201312152278533374192131947784457009467221420830916539319728138153456828438